In [45]:
import numpy as np
import torch
from torch import nn
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Running on",device)

In [51]:
dat = np.genfromtxt("mnist_train.csv", delimiter = ",", skip_header = 1)

In [61]:
X, y = dat[:,1:], dat[:,0]

X = torch.tensor(X, requires_grad=True )

y = torch.tensor(y, requires_grad=False, dtype = int)

In [75]:
def batch_loader(X,y, batch_size = 6):
    for i in range(y.shape[0]//batch_size):
        yield X[i*batch_size:(i+1)*batch_size], y[i*batch_size:(i+1)*batch_size]

In [81]:
class CNN(nn.Module):
    def __init__(self,):
        super (CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(2, 2))
        self.activation1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))
        self.activation2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.linear1 = nn.Linear(in_features=800, out_features=500)
        self.activation3 = nn.ReLU()

        self.linear2 = nn.Linear(in_features=500, out_features=9)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view((28,28))
        x.unsqueeze(1)
        print(x.shape)
        x = self.activation1(self.conv1(x))
        x = self.maxpool1(x)

        x = self.activation2(self.conv2(x))
        x = self.maxpool2(x)

        x = self.activation3(self.linear1(x))
        x = self.linear2(x)

        out = self.logSoftmax(x)

        return out


In [84]:
model = CNN()

optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

def train(n_epochs):
    model.train()

    for i in range(n_epochs):
        Loader = batch_loader(X,y)

        c = 0
        epoch_loss = 0
        epoch_acc = 0
        for dat, label in Loader:
            pred = model(dat)

            loss = loss_fn(label, pred)

            loss.backward()
            optimizer.step()

            epoch_loss += loss
            epoch_acc += (label == pred).mean()

        print(f"Epoch {i} - loss {epoch_loss/c:.3} - acc {epoch_acc/c:.3}")


In [85]:
train(10)

RuntimeError: shape '[28, 28]' is invalid for input of size 4704